In [1]:
import numpy as np
import tkinter as tk
from tkinter import filedialog
import yaml
from yaml.loader import SafeLoader
import os
import glob
import os
import sys
import tifffile

SCRIPT_DIR = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
sys.path.append(os.path.dirname(SCRIPT_DIR))
from src.processing.processing_functions import *
from src.train_engine.models import *

# get working directory
path = os.getcwd()
sys.path.append(path)

# import machine learning libraries
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, utils
from monai.inferers.inferer import SlidingWindowInferer, SliceInferer
from monai.networks.nets import BasicUNet, UNet
from monai.transforms import (
    Activations,
    AsDiscrete,
    Compose,
)

## Load YAML Config File

In [12]:
yaml_file = f"C:\\Users\\Fungj\\Google Drive\\Masters Project\\Segmentation-Model\\3D-Neuron-Segment\\config\\transformers\\SwinUNetR\\kfold_training\\soma_dendrite.yml"

In [13]:
with open(yaml_file) as f:
    config = yaml.load(f, Loader=SafeLoader)
    print(config)

{'model_name': 'ResUNet', 'parent_dir': '/home/haas/projects/def-haas/jsfung', 'raw_path': '/home/jsfung/projects/def-haas/jsfung/Images/new_labels/Raw/*.tif', 'mask_path': '/home/jsfung/projects/def-haas/jsfung/Images/new_labels/Mask/*.tif', 'date': 'now', 'DATASET': {'exp': '+s_+d_-f', 'batch_size': 128, 'lateral_steps': 64, 'axial_steps': 8, 'z_patch': 32, 'y_patch': 128, 'x_patch': 128, 'folds': 4, 'remove_artifacts': False, 'artifacts': [7], 'ex_autofluorescence': False, 'ex_melanocytes': True, 'AUGMENTATION': {'augment': True, 'z_deg': 25, 'y_deg': 0, 'x_deg': 0, 'gamma_lower': -0.5, 'gamma_upper': 0.6, 'mean_noise': 0.05, 'std_noise': 0.025}}, 'loss': 'dice_ce', 'weights': None, 'max_epochs': 300, 'MODEL': {'model_arch': 'SwinUNETR', 'dropout': 0.1, 'attn_dropout': 0.1, 'dropout_path_rate': 0.1, 'learning_rate': 7.54e-06, 'l2': 0.00421, 'norm': 'instance', 'input_dim': 1, 'depths': [2, 2, 2], 'num_heads': [6, 12, 24], 'feature_size': 24, 'spatial_dim': 3}, 'end_cycle': 30, 'TRAI

In [14]:
model = Build_Model(config, num_classes=4)

In [15]:
model.parameters

<bound method Module.parameters of SwinUNETR(
  (swinViT): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(1, 24, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (pos_drop): Dropout(p=0.1, inplace=False)
    (layers1): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=24, out_features=72, bias=True)
              (attn_drop): Dropout(p=0.1, inplace=False)
              (proj): Linear(in_features=24, out_features=24, bias=True)
              (proj_drop): Dropout(p=0.1, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
            (mlp): MLPBlock(
              (linear1): Linear(in_features=24, out_features=96, bias=True)
              (

In [10]:
import monai
img_size = (32,128,128)
patch_size = monai.utils.misc.ensure_tuple_rep(2,3)

for m, p in zip(img_size, patch_size):
    for i in range(5):
        print(np.power(p,i+1))

        if m % np.power(p, i + 1) != 0:
            print("invalid")

2
4
8
16
32
2
4
8
16
32
2
4
8
16
32


In [16]:
from torchsummary import summary

In [20]:
summary(model, (1,32,128,128))

IndexError: index 0 is out of range